In [1]:
CODE_DIR = '/home/shreyas/Documents/git/wheelai/'
traindata_path = '/media/shreyas/DATA/ML_DATA/wheelai/gtaV/sample_train/'
validdata_path = '/media/shreyas/DATA/ML_DATA/wheelai/gtaV/valid/'
results_path = '/media/shreyas/DATA/ML_DATA/wheelai/gtaV/results/'

In [42]:
from __future__ import division, print_function

import os, json
from glob import glob
import numpy as np
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom

import keras
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential, load_model,  Model
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.pooling import GlobalAveragePooling2D
from keras.optimizers import Adam, SGD
from keras.preprocessing import image
from keras.applications import VGG16
from keras.layers import Input, Lambda, Cropping2D, Activation, ELU
from keras.layers.merge import add, concatenate

%matplotlib inline

### Finetune and Train

In [3]:
batch_size = 64
image_dims = (160,320)

In [4]:
def get_batches(path, class_mode='categorical', gen=image.ImageDataGenerator(), \
                shuffle=True, target_size=image_dims, batch_size=1):
    return gen.flow_from_directory(path, class_mode=class_mode, batch_size=batch_size, \
                                   target_size=target_size, shuffle=shuffle)

def get_steps(batches, batch_size):
    steps = int(batches.samples/batch_size)
    return (steps if batches.samples%batch_size==0 else (steps+1))


In [5]:
train_b = get_batches(traindata_path, batch_size=batch_size)
valid_b = get_batches(validdata_path, batch_size=batch_size)

Found 12954 images belonging to 3 classes.
Found 3375 images belonging to 3 classes.


In [6]:
train_steps = get_steps(train_b, batch_size)
valid_steps = get_steps(valid_b, batch_size)
num_class = train_b.num_class

In [7]:
train_labels = train_b.classes
valid_labels = valid_b.classes

In [8]:
y_train = keras.utils.to_categorical(train_labels)
y_valid = keras.utils.to_categorical(valid_labels)

In [9]:
print(train_labels.shape, valid_labels.shape, y_train.shape, y_valid.shape)

(12954,) (3375,) (12954, 3) (3375, 3)


In [10]:
trn_b = get_batches(traindata_path, class_mode=None, shuffle=False, batch_size=batch_size)
val_b = get_batches(validdata_path, class_mode=None, shuffle=False, batch_size=batch_size)

Found 12954 images belonging to 3 classes.
Found 3375 images belonging to 3 classes.


In [11]:
gen_augdata = image.ImageDataGenerator(width_shift_range=.2, height_shift_range=.2, 
                                      shear_range=0.2, zoom_range=0.2)
augtrain_b = get_batches(traindata_path, gen=gen_augdata, class_mode='categorical',\
                         batch_size=batch_size)

Found 12954 images belonging to 3 classes.


In [12]:
def nvidia_model():
    x = Input(shape=(160, 320, 3))
    crop_im = Cropping2D(cropping=((70,0), (0,0)))(x)
    
    color_s = Conv2D(3, (1,1), padding='same')(crop_im)
    norm = Lambda(lambda x: (x / 255.0) - 0.5)(color_s)
    
    conv_b1 = Conv2D(24, (3,3), padding='same', activation='relu')(norm)
    batch_1 = BatchNormalization()(conv_b1)
    max_pl1 = MaxPooling2D()(batch_1)
    
    conv_b2 = Conv2D(36, (3,3), padding='same', activation='relu')(max_pl1)
    batch_2 = BatchNormalization()(conv_b2)
    max_pl2 = MaxPooling2D()(batch_2)
    
    conv_b3 = Conv2D(48, (3,3), padding='same', activation='relu')(max_pl2)
    batch_3 = BatchNormalization()(conv_b3)
    max_pl3 = MaxPooling2D()(batch_3)
    
    conv_b4 = Conv2D(64, (3,3), padding='same', activation='relu')(max_pl3)
    batch_4 = BatchNormalization()(conv_b4)
    max_pl4 = MaxPooling2D()(batch_4)
    dropout = Dropout(0.1)(max_pl4)
    
    conv_b5 = Conv2D(64, (3,3), padding='same', activation='relu')(dropout)
    batch_5 = BatchNormalization()(conv_b5)
    max_pl5 = MaxPooling2D((1,4))(batch_5)
    dropout = Dropout(0.1)(max_pl5)
    
    flatten = Flatten()(dropout)
    fc1 = Dense(100, activation='relu')(flatten)
    batch_fc1 = BatchNormalization()(fc1)
    drop_fc1 = Dropout(0.2)(batch_fc1)
    
    fc2 = Dense(50, activation='relu')(drop_fc1)
    batch_fc2 = BatchNormalization()(fc2)
    drop_fc2 = Dropout(0.3)(batch_fc2)
    
    fc3 = Dense(10, activation='relu')(drop_fc2)
    batch_fc3 = BatchNormalization()(fc3)
    drop_fc3 = Dropout(0.5)(batch_fc3)
    
    output = Dense(3, activation='softmax')(drop_fc3)
    
    model = Model(inputs=x, outputs=output)
    return model

In [53]:
def nvidia_conv():
    model = Sequential()
    model.add(Cropping2D(cropping=((70,0), (0,0)), input_shape=(160, 320, 3)))
    model.add(Lambda(lambda x: (x / 255.0) - 0.5))
    model.add(Conv2D(3, (1,1), padding='same')) 
    model.add(Conv2D(24, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D())
    model.add(Conv2D(36, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D())   
    model.add(Conv2D(48, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D())
    model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D())
    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((1,4)))
    model.add(Conv2D(3, (3,3), padding='same'))
    model.add(GlobalAveragePooling2D())
    model.add(Activation('softmax'))
    
    return model

In [43]:
def base_model():
    model = Sequential()
    model.add(Cropping2D(cropping=((70,0), (0,0)), input_shape=(160, 320, 3)))
    model.add(Lambda(lambda x: (x / 255.0) - 0.5))
    model.add(Conv2D(3, (1,1), padding='same')) 
    model.add(Conv2D(32, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D())
    model.add(Dropout(0.1))
    model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D())
    model.add(Dropout(0.1))
    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D())
    model.add(Dropout(0.2))
    model.add(Flatten())    
    model.add(Dense(256, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(256, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(3, activation='softmax'))
    return model

In [70]:
def amazon_model():
    x = Input(shape = (160, 320, 3))
    crop = Cropping2D(cropping=((70,0), (0,0)), input_shape=(160, 320, 3))(x)
    norm = Lambda(lambda x: (x / 255.0) - 0.5)(crop)
    
    prep = Conv2D(10, (1,1), padding='same', activation='elu')(norm)
    prep = Conv2D(10, (1,1), padding='same', activation='elu')(prep)
    prep = Conv2D(3, (1,1), padding='same', activation='elu')(prep)
    
    conv1 = Conv2D(16, (3,3), padding='same', activation='relu')(prep)
    norm1 = BatchNormalization()(conv1)
    conv1 = Conv2D(32, (1,1), padding='same', activation='relu')(norm1)
    norm1 = BatchNormalization()(conv1)
    conv1 = Conv2D(32, (1,1), padding='same', activation='relu')(norm1)
    norm1 = BatchNormalization()(conv1)
    conv1 = Conv2D(32, (3,3), padding='same', activation='relu')(norm1)
    norm1 = BatchNormalization()(conv1)
    max_1 = MaxPooling2D()(norm1)
    
    conv2 = Conv2D(32, (3,3), padding='same', activation='relu')(max_1)
    norm2 = BatchNormalization()(conv2)
    conv2 = Conv2D(48, (1,1), padding='same', activation='relu')(norm2)
    norm2 = BatchNormalization()(conv2)
    conv2 = Conv2D(48, (1,1), padding='same', activation='relu')(norm2)
    norm2 = BatchNormalization()(conv2)
    conv2 = Conv2D(48, (3,3), padding='same', activation='relu')(norm2)
    norm2 = BatchNormalization()(conv2)
    max_2 = MaxPooling2D()(norm2)
    gavg1 = GlobalAveragePooling2D()(max_2)
    
    conv3 = Conv2D(48, (3,3), padding='same', activation='relu')(max_2)
    norm3 = BatchNormalization()(conv3)
    conv3 = Conv2D(64, (1,1), padding='same', activation='relu')(norm3)
    norm3 = BatchNormalization()(conv3)
    conv3 = Conv2D(64, (1,1), padding='same', activation='relu')(norm3)
    norm3 = BatchNormalization()(conv3)
    conv3 = Conv2D(64, (3,3), padding='same', activation='relu')(norm3)
    norm3 = BatchNormalization()(conv3)
    max_3 = MaxPooling2D()(norm3)
    gavg2 = GlobalAveragePooling2D()(max_3)
    
    conv4 = Conv2D(64, (3,3), padding='same', activation='relu')(max_3)
    norm4 = BatchNormalization()(conv4)
    conv4 = Conv2D(128, (1,1), padding='same', activation='relu')(norm4)
    norm4 = BatchNormalization()(conv4)
    conv4 = Conv2D(128, (1,1), padding='same', activation='relu')(norm4)
    norm4 = BatchNormalization()(conv4)
    conv4 = Conv2D(128, (3,3), padding='same', activation='relu')(norm4)
    norm4 = BatchNormalization()(conv4)
    max_4 = MaxPooling2D()(norm4)
    gavg3 = GlobalAveragePooling2D()(max_4)
    
    merged = concatenate([gavg1, gavg2, gavg3])
    fc_1 = Dense(240, activation='relu')(merged)
    norm = BatchNormalization()(fc_1)
    dropout = Dropout(0.5)(norm)
    fc_2 = Dense(120, activation='relu')(dropout)
    norm = BatchNormalization()(fc_2)
    dropout = Dropout(0.5)(norm)
    outout = Dense(3, activation='softmax')(dropout)
    
    model = Model(inputs=x, outputs=outout)
    
    return model

## 1 Nvidia Model

In [15]:
model = nvidia_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 160, 320, 3)       0         
_________________________________________________________________
cropping2d_1 (Cropping2D)    (None, 90, 320, 3)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 90, 320, 3)        12        
_________________________________________________________________
lambda_1 (Lambda)            (None, 90, 320, 3)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 90, 320, 24)       672       
_________________________________________________________________
batch_normalization_1 (Batch (None, 90, 320, 24)       96        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 45, 160, 24)       0         
__________

In [16]:
adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(optimizer=adam,loss='categorical_crossentropy')

In [17]:
#filepath1 = results_path+'nvidia.h5'
#checkpoint1 = ModelCheckpoint(filepath1, monitor='val_loss', verbose=1, save_best_only=False,\
#                             save_weights_only=True, mode='min', period=1)
#callbacks1=[checkpoint1]

In [19]:
model.fit_generator(train_b, steps_per_epoch=train_steps, epochs=2, verbose=1,\
                        validation_data=valid_b, validation_steps=valid_steps)

model.save_weights(results_path+'nvidia(1e-4).h5')

Epoch 1/2
203/203 [==============================] - 174s - loss: 1.7997 - val_loss: 1.2216
Epoch 2/2
203/203 [==============================] - 167s - loss: 1.4813 - val_loss: 1.1355


In [20]:
model.optimizer.lr = 1e-5
model.fit_generator(train_b, steps_per_epoch=train_steps, epochs=3, verbose=1,\
                        validation_data=valid_b, validation_steps=valid_steps)

model.save_weights(results_path+'nvidia(1e-5).h5')

Epoch 1/3
203/203 [==============================] - 174s - loss: 1.3393 - val_loss: 0.9874
Epoch 2/3
203/203 [==============================] - 168s - loss: 1.2671 - val_loss: 1.0061
Epoch 3/3
203/203 [==============================] - 167s - loss: 1.1828 - val_loss: 0.9705


In [22]:
model.optimizer.lr = 1e-3
model.fit_generator(train_b, steps_per_epoch=train_steps, epochs=1, verbose=1,\
                        validation_data=valid_b, validation_steps=valid_steps)

model.save_weights(results_path+'nvidia(1e-3).h5')

Epoch 1/1
203/203 [==============================] - 173s - loss: 1.1232 - val_loss: 0.9599


In [ ]:
#filepath2 = results_path+'nvidia_aug.h5'
#checkpoint2 = ModelCheckpoint(filepath2, monitor='val_loss', verbose=1, save_best_only=False,\
#                             save_weights_only=True, mode='min', period=1)
#callbacks2=[checkpoint2]

In [23]:
model.optimizer.lr = 1e-4
model.fit_generator(augtrain_b, steps_per_epoch=train_steps, epochs=3, verbose=1,\
                        validation_data=valid_b, validation_steps=valid_steps)

model.save_weights(results_path+'nvidia_aug(1e-4).h5')

Epoch 1/3
203/203 [==============================] - 227s - loss: 1.3644 - val_loss: 1.0323
Epoch 2/3
203/203 [==============================] - 226s - loss: 1.2811 - val_loss: 1.0373
Epoch 3/3
203/203 [==============================] - 217s - loss: 1.2287 - val_loss: 1.0467


In [24]:
model.optimizer.lr = 1e-3
model.fit_generator(augtrain_b, steps_per_epoch=train_steps, epochs=2, verbose=1,\
                        validation_data=valid_b, validation_steps=valid_steps)

model.save_weights(results_path+'nvidia_aug(1e-3).h5')

Epoch 1/2
203/203 [==============================] - 238s - loss: 1.1842 - val_loss: 1.0170
Epoch 2/2
203/203 [==============================] - 228s - loss: 1.1599 - val_loss: 1.0724


In [25]:
model.optimizer.lr = 1e-5
model.fit_generator(augtrain_b, steps_per_epoch=train_steps, epochs=2, verbose=1,\
                        validation_data=valid_b, validation_steps=valid_steps)

model.save_weights(results_path+'nvidia_aug(1e-5).h5')

Epoch 1/2
203/203 [==============================] - 234s - loss: 1.1302 - val_loss: 1.0188
Epoch 2/2
203/203 [==============================] - 225s - loss: 1.1092 - val_loss: 1.0169


## 2 Nvidia All CONV model

In [54]:
model = nvidia_conv()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cropping2d_8 (Cropping2D)    (None, 90, 320, 3)        0         
_________________________________________________________________
lambda_8 (Lambda)            (None, 90, 320, 3)        0         
_________________________________________________________________
conv2d_36 (Conv2D)           (None, 90, 320, 3)        12        
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 90, 320, 24)       672       
_________________________________________________________________
batch_normalization_32 (Batc (None, 90, 320, 24)       96        
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 45, 160, 24)       0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 45, 160, 36)       7812      
__________

In [55]:
adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(optimizer=adam,loss='categorical_crossentropy')

In [56]:
model.fit_generator(train_b, steps_per_epoch=train_steps, epochs=2, verbose=1,\
                        validation_data=valid_b, validation_steps=valid_steps)

model.save_weights(results_path+'nvidia_conv(1e-4).h5')

Epoch 1/2
203/203 [==============================] - 172s - loss: 1.0049 - val_loss: 1.1923
Epoch 2/2
203/203 [==============================] - 164s - loss: 0.8197 - val_loss: 1.3432


In [57]:
model.optimizer.lr = 1e-5
model.fit_generator(train_b, steps_per_epoch=train_steps, epochs=3, verbose=1,\
                        validation_data=valid_b, validation_steps=valid_steps)

model.save_weights(results_path+'nvidia_conv(1e-5).h5')

Epoch 1/3
203/203 [==============================] - 172s - loss: 0.7454 - val_loss: 0.9915
Epoch 2/3
203/203 [==============================] - 166s - loss: 0.6958 - val_loss: 0.8565
Epoch 3/3
203/203 [==============================] - 165s - loss: 0.6637 - val_loss: 0.9553


In [58]:
model.optimizer.lr = 1e-3
model.fit_generator(train_b, steps_per_epoch=train_steps, epochs=1, verbose=1,\
                        validation_data=valid_b, validation_steps=valid_steps)

model.save_weights(results_path+'nvidia_conv(1e-3).h5')

Epoch 1/1
203/203 [==============================] - 171s - loss: 0.6396 - val_loss: 0.8525


In [59]:
#filepath2 = results_path+'nvidia_aug.h5'
#checkpoint2 = ModelCheckpoint(filepath2, monitor='val_loss', verbose=1, save_best_only=False,\
#                             save_weights_only=True, mode='min', period=1)
#callbacks2=[checkpoint2]

In [60]:
model.optimizer.lr = 1e-4
model.fit_generator(augtrain_b, steps_per_epoch=train_steps, epochs=3, verbose=1,\
                        validation_data=valid_b, validation_steps=valid_steps)

model.save_weights(results_path+'nvidia_conv_aug(1e-4).h5')

Epoch 1/3
203/203 [==============================] - 226s - loss: 0.9572 - val_loss: 0.9619
Epoch 2/3
203/203 [==============================] - 217s - loss: 0.9010 - val_loss: 1.0469
Epoch 3/3
203/203 [==============================] - 217s - loss: 0.9013 - val_loss: 1.0190


In [61]:
model.optimizer.lr = 1e-3
model.fit_generator(augtrain_b, steps_per_epoch=train_steps, epochs=2, verbose=1,\
                        validation_data=valid_b, validation_steps=valid_steps)

model.save_weights(results_path+'nvidia_conv_aug(1e-3).h5')

Epoch 1/2
203/203 [==============================] - 225s - loss: 0.8762 - val_loss: 0.9665
Epoch 2/2
203/203 [==============================] - 216s - loss: 0.8772 - val_loss: 0.9583


In [62]:
model.optimizer.lr = 1e-5
model.fit_generator(augtrain_b, steps_per_epoch=train_steps, epochs=2, verbose=1,\
                        validation_data=valid_b, validation_steps=valid_steps)

model.save_weights(results_path+'nvidia_conv_aug(1e-5).h5')

Epoch 1/2
203/203 [==============================] - 226s - loss: 0.8612 - val_loss: 1.0659
Epoch 2/2
203/203 [==============================] - 216s - loss: 0.8528 - val_loss: 1.0240


## 3 Amazon Model

In [71]:
model = amazon_model()
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_6 (InputLayer)             (None, 160, 320, 3)   0                                            
____________________________________________________________________________________________________
cropping2d_12 (Cropping2D)       (None, 90, 320, 3)    0                                            
____________________________________________________________________________________________________
lambda_12 (Lambda)               (None, 90, 320, 3)    0                                            
____________________________________________________________________________________________________
conv2d_100 (Conv2D)              (None, 90, 320, 10)   40                                           
___________________________________________________________________________________________

In [72]:
adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(optimizer=adam,loss='categorical_crossentropy')

In [73]:
model.fit_generator(train_b, steps_per_epoch=train_steps, epochs=2, verbose=1,\
                        validation_data=valid_b, validation_steps=valid_steps)

model.save_weights(results_path+'amazon(1e-4).h5')

Epoch 1/2
203/203 [==============================] - 304s - loss: 1.7531 - val_loss: 1.1207
Epoch 2/2
203/203 [==============================] - 303s - loss: 1.5971 - val_loss: 1.4024


In [74]:
model.optimizer.lr = 1e-5
model.fit_generator(train_b, steps_per_epoch=train_steps, epochs=3, verbose=1,\
                        validation_data=valid_b, validation_steps=valid_steps)

model.save_weights(results_path+'amazon(1e-5).h5')

Epoch 1/3
203/203 [==============================] - 306s - loss: 1.5257 - val_loss: 1.1276
Epoch 2/3
203/203 [==============================] - 305s - loss: 1.4689 - val_loss: 1.1711
Epoch 3/3
203/203 [==============================] - 303s - loss: 1.4199 - val_loss: 1.1248


In [75]:
model.optimizer.lr = 1e-3
model.fit_generator(train_b, steps_per_epoch=train_steps, epochs=1, verbose=1,\
                        validation_data=valid_b, validation_steps=valid_steps)

model.save_weights(results_path+'amazon(1e-3).h5')

Epoch 1/1
203/203 [==============================] - 303s - loss: 1.3843 - val_loss: 1.1659


In [76]:
#filepath2 = results_path+'nvidia_aug.h5'
#checkpoint2 = ModelCheckpoint(filepath2, monitor='val_loss', verbose=1, save_best_only=False,\
#                             save_weights_only=True, mode='min', period=1)
#callbacks2=[checkpoint2]

In [77]:
model.optimizer.lr = 1e-4
model.fit_generator(augtrain_b, steps_per_epoch=train_steps, epochs=3, verbose=1,\
                        validation_data=valid_b, validation_steps=valid_steps)

model.save_weights(results_path+'amazon_aug(1e-4).h5')

Epoch 1/3
203/203 [==============================] - 303s - loss: 1.3432 - val_loss: 1.1212
Epoch 2/3
203/203 [==============================] - 303s - loss: 1.3103 - val_loss: 1.2252
Epoch 3/3
203/203 [==============================] - 302s - loss: 1.2807 - val_loss: 1.2489


In [78]:
model.optimizer.lr = 1e-3
model.fit_generator(augtrain_b, steps_per_epoch=train_steps, epochs=2, verbose=1,\
                        validation_data=valid_b, validation_steps=valid_steps)

model.save_weights(results_path+'amazon_aug(1e-3).h5')

Epoch 1/2
203/203 [==============================] - 306s - loss: 1.2456 - val_loss: 1.7032
Epoch 2/2
203/203 [==============================] - 308s - loss: 1.2525 - val_loss: 1.1948


In [79]:
model.optimizer.lr = 1e-5
model.fit_generator(augtrain_b, steps_per_epoch=train_steps, epochs=2, verbose=1,\
                        validation_data=valid_b, validation_steps=valid_steps)

model.save_weights(results_path+'amazon_aug(1e-5).h5')

Epoch 1/2
203/203 [==============================] - 303s - loss: 1.2140 - val_loss: 1.1525
Epoch 2/2
203/203 [==============================] - 304s - loss: 1.2020 - val_loss: 1.1098


## 4 Base Model

In [44]:
model = base_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cropping2d_7 (Cropping2D)    (None, 90, 320, 3)        0         
_________________________________________________________________
lambda_7 (Lambda)            (None, 90, 320, 3)        0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 90, 320, 3)        12        
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 90, 320, 32)       896       
_________________________________________________________________
batch_normalization_27 (Batc (None, 90, 320, 32)       128       
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 45, 160, 32)       0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 45, 160, 32)       0         
__________

In [45]:
adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(optimizer=adam,loss='categorical_crossentropy')

In [46]:
model.fit_generator(train_b, steps_per_epoch=train_steps, epochs=2, verbose=1,\
                        validation_data=valid_b, validation_steps=valid_steps)

model.save_weights(results_path+'base(1e-4).h5')

Epoch 1/2
203/203 [==============================] - 177s - loss: 1.3846 - val_loss: 2.1978
Epoch 2/2
203/203 [==============================] - 164s - loss: 1.1824 - val_loss: 1.9703


In [47]:
model.optimizer.lr = 1e-5
model.fit_generator(train_b, steps_per_epoch=train_steps, epochs=3, verbose=1,\
                        validation_data=valid_b, validation_steps=valid_steps)

model.save_weights(results_path+'base(1e-5).h5')

Epoch 1/3
203/203 [==============================] - 171s - loss: 1.0318 - val_loss: 1.0129
Epoch 2/3
203/203 [==============================] - 165s - loss: 0.9224 - val_loss: 0.8906
Epoch 3/3
203/203 [==============================] - 165s - loss: 0.7950 - val_loss: 0.9065


In [48]:
model.optimizer.lr = 1e-3
model.fit_generator(train_b, steps_per_epoch=train_steps, epochs=1, verbose=1,\
                        validation_data=valid_b, validation_steps=valid_steps)

model.save_weights(results_path+'base(1e-3).h5')

Epoch 1/1
203/203 [==============================] - 170s - loss: 0.7074 - val_loss: 0.9304


In [49]:
#filepath2 = results_path+'nvidia_aug.h5'
#checkpoint2 = ModelCheckpoint(filepath2, monitor='val_loss', verbose=1, save_best_only=False,\
#                             save_weights_only=True, mode='min', period=1)
#callbacks2=[checkpoint2]

In [50]:
model.optimizer.lr = 1e-4
model.fit_generator(augtrain_b, steps_per_epoch=train_steps, epochs=3, verbose=1,\
                        validation_data=valid_b, validation_steps=valid_steps)

model.save_weights(results_path+'base_aug(1e-4).h5')

Epoch 1/3
203/203 [==============================] - 227s - loss: 1.3445 - val_loss: 1.5227
Epoch 2/3
203/203 [==============================] - 217s - loss: 1.2114 - val_loss: 1.2137
Epoch 3/3
203/203 [==============================] - 216s - loss: 1.1757 - val_loss: 1.0981


In [51]:
model.optimizer.lr = 1e-3
model.fit_generator(augtrain_b, steps_per_epoch=train_steps, epochs=2, verbose=1,\
                        validation_data=valid_b, validation_steps=valid_steps)

model.save_weights(results_path+'base_aug(1e-3).h5')

Epoch 1/2
203/203 [==============================] - 225s - loss: 1.1086 - val_loss: 1.0019
Epoch 2/2
203/203 [==============================] - 217s - loss: 1.0737 - val_loss: 1.0344


In [52]:
model.optimizer.lr = 1e-5
model.fit_generator(augtrain_b, steps_per_epoch=train_steps, epochs=2, verbose=1,\
                        validation_data=valid_b, validation_steps=valid_steps)

model.save_weights(results_path+'base_aug(1e-5).h5')

Epoch 1/2
203/203 [==============================] - 226s - loss: 1.0273 - val_loss: 1.0051
Epoch 2/2
203/203 [==============================] - 218s - loss: 1.0255 - val_loss: 1.0045


#serialize model to JSON
model_path = results_path+'nvidia.json'
model_json = model.to_json()
with open(model_path, "w") as json_file:
    json_file.write(model_json)
#serialize weights to HDF5
model.save_weights(filepath2)
print("Saved model to disk")